# Debug the Linux Kernel Using Ftrace

- [Setting up Ftrace](#setup_ftrace)  
- [Basic Operatoins](#basic_operation)  
-- [Tracers available](#available_tracers)  
-- [Enable/Disable Ftrace](#enable_disable)  
-- [Show/Clear Trace Messages](#show_clear_msg)  
-- [Using `trace_printk()`](#using_trace_printk)  
-- [Ring Buffer Overflow Problem](#ring_buff_overflow)  
-- [Starting and Stopping the Trace Recording](#start_stop_recording)

Note: The commands within this article must be run as the root user (`sudo` is insufficient). So, if we need to re-execute this `.ipynb`, we can used this command to start up the jupyter lab:
```sh
sudo -E env "PATH=$PATH" jupyter lab --allow-root
```

## <a id='setup_ftrace'>1. Setting up Ftrace</a>

- 1.1 For the purpose of debugging, the kernel configuration parameters that should be enabled are:

In [1]:
for config in "CONFIG_FUNCTION_TRACER" "CONFIG_FUNCTION_GRAPH_TRACER" \
              "CONFIG_STACK_TRACER" "ONFIG_DYNAMIC_FTRACE"
do
    grep "${config}\b" /boot/config-$(uname -r)
done

CONFIG_FUNCTION_TRACER=y
CONFIG_FUNCTION_GRAPH_TRACER=y
CONFIG_STACK_TRACER=y
CONFIG_DYNAMIC_FTRACE=y


- 1.2 Currently the API to interface with Ftrace is located in the Debugfs file system. Typically, that is mounted at `/sys/kernel/debug`. So, the first step to check the ftrace setting is to verify the existence of the filesystem.

In [2]:
mount | grep -i debugfs

debugfs on /sys/kernel/debug type debugfs (rw,nosuid,nodev,noexec,relatime)


Let's define a function to find the debugfs:

In [6]:
function find_debugfs()
{
    mount | awk '/^debugfs on .* type debugfs/{print $3}'
}

DEBUGFS=$(find_debugfs); echo "debugfs is at: $DEBUGFS"

debugfs is at: /sys/kernel/debug


- 1.3 When Ftrace is configured, it will create its own directory called *tracing* within the Debugfs file system.

In [4]:
ls $DEBUGFS/tracing

available_events            max_graph_depth         stack_max_size
available_filter_functions  options                 stack_trace
available_tracers           per_cpu                 stack_trace_filter
buffer_percent              printk_formats          synthetic_events
buffer_size_kb              README                  timestamp_mode
buffer_total_size_kb        saved_cmdlines          trace
current_tracer              saved_cmdlines_size     trace_clock
dynamic_events              saved_tgids             trace_marker
dyn_ftrace_total_info       set_event               trace_marker_raw
enabled_functions           set_event_notrace_pid   trace_options
error_log                   set_event_pid           trace_pipe
events                      set_ftrace_filter       trace_stat
free_buffer                 set_ftrace_notrace      tracing_cpumask
function_profile_enabled    set_ftrace_notrace_pid  tracing_max_latency
hwlat_detector              set_ftrace_pid          tracing_on
instances  

## <a id='basic_operation'> 2. Basic Operatoins</a>

### <a id='available_tracers'>2.1 Tracers available</a>
To find out which tracers are available, simply cat the *available_tracers* file in the tracing directory:

In [5]:
cat $DEBUGFS/tracing/available_tracers

hwlat blk mmiotrace function_graph wakeup_dl wakeup_rt wakeup function nop


The tracer `nop` is short for `no operation`, is used to disable the trace.

We can check the current tracer in the file *current_tracer*:

In [6]:
cat $DEBUGFS/tracing/current_tracer

nop


### <a id='enable_disable'>2.2 Enable/Disable a Tracer</a>
To enable the `function` tracer, just echo "function" into the current_tracer file.

In [7]:
# set current tracer to "function"
echo "function" > $DEBUGFS/tracing/current_tracer

In [8]:
# show current tracer
cat $DEBUGFS/tracing/current_tracer

function


In [9]:
# disable tracer
echo "nop" > $DEBUGFS/tracing/current_tracer

### <a id='show_clear_msg'> 2.3 Show/Clear Trace Messages</a>

We can get trace information through this file: *trace*. We can read the message with any tools. To clear the message, just write an empty string into the file:

In [18]:
echo "function" > $DEBUGFS/tracing/current_tracer # enable `function` tracer
sleep 1; # let the bullet fly...
head -n 20 $DEBUGFS/tracing/trace # read messages
echo "nop" > $DEBUGFS/tracing/current_tracer # disable

# tracer: function
#
# entries-in-buffer/entries-written: 409969/1780364   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |
          <idle>-0       [002] d..2.  6530.912059: ktime_get <-tick_nohz_irq_exit
          <idle>-0       [002] ...2.  6530.912061: cpuidle_reflect <-do_idle
          <idle>-0       [002] ...2.  6530.912061: menu_reflect <-cpuidle_reflect
          <idle>-0       [002] ...2.  6530.912061: tick_nohz_idle_got_tick <-menu_reflect
          <idle>-0       [002] ...2.  6530.912062: arch_cpu_idle_exit <-do_idle
          <idle>-0       [002] d..2.  6530.912062: arch_cp

In [11]:
echo > $DEBUGFS/tracing/trace # clear messages
cat $DEBUGFS/tracing/trace

# tracer: nop
#
# entries-in-buffer/entries-written: 0/0   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |


The meaning of the headers within `trace` file is quite straightforeward:
- TASK: the name of process.
- PID: process id.
- CPU: CPU number.
- TIMESTAMP: the time since boot, in second.
- FUNCTION: the function captured, followed by its parent.

The `function_graph` serves the same purpose but it shows information in a friendly way for humans:

In [12]:
echo 'function_graph' > $DEBUGFS/tracing/current_tracer # enable tracer
head -n 30 $DEBUGFS/tracing/trace # read

echo > $DEBUGFS/tracing/trace # clear
echo "nop" > $DEBUGFS/tracing/current_tracer # disable

# tracer: function_graph
#
# CPU  DURATION                  FUNCTION CALLS
# |     |   |                     |   |   |   |
 7)   0.180 us    |        } /* tick_nohz_tick_stopped */
 7)   9.154 us    |      } /* menu_select */
 7)   9.527 us    |    } /* cpuidle_select */
 7)   0.186 us    |    tick_nohz_tick_stopped();
 7)               |    tick_nohz_idle_retain_tick() {
 7)   0.187 us    |      timer_clear_idle();
 7)   0.565 us    |    }
 7)               |    cpuidle_enter() {
 7)   0.193 us    |      tick_nohz_get_next_hrtimer();
 7)               |      cpuidle_enter_state() {
 7)   0.134 us    |        sched_idle_set_state();
 7)   0.916 us    |        intel_idle();
 7)   0.197 us    |        sched_idle_set_state();
 7)               |        irq_enter_rcu() {
 7)               |          tick_irq_enter() {
 7)   0.192 us    |            tick_check_oneshot_broadcast_this_cpu();
 7)   0.220 us    |            ktime_get();
 7)   0.189 us    |            nr_iowait_cpu();
 7)   1.34

The header **DURATION** is in microseconds(10e-6 second).  
The *"!"* sign in *"7) ! 825.996 us  |        intel_idle();"* is a marker annotating *"greater than 100 micorseconds"*. Similarly, a *"+"* annotates *"greater than 10 microseconds"*.

Before we continue, let's solve a problem I encountered: when the current tracer is `function` or `function_grapth`, command `cat trace` will never stop on my laptop! Let's impletement a tool to solve this problem: 

In [4]:
# @desc - grep function name from /sys/kernel/debug/tracing/trace
# @usage - filter_trace <pattern> [max match count] [max lines to filter]
function filter_trace() (
    REGX=${1:?"no regx"}  # regular expression
    MAXMATCH=${2:-3}      # max match count
    MAXLINES=${3:-1000}   # max number record

    awk -v pattern="$REGX" -v max_match="$MAXMATCH" -v max_lines="$MAXLINES" '
        function enque_ring_buff(ring_buff) {
            ring_buff[FNR] = $0
            if (FNR-5 in ring_buff) {
                delete ring_buff[FNR-5]; n_ignore += 1
            }
        }
        function print_clear_ring_buff(ring_buff) {
            if (n_ignore > 0) {
                print " >>>>>>>>>>>>>>>>> "n_ignore" lines ignored <<<<<<<<<<<"
                n_ignore = 0
            }
            for (nr in ring_buff) {
                print ring_buff[nr]
            }
            delete ring_buff
        }

        BEGIN {
            n_match   = 0                     # number of matched
            n_headers = 12                    # 12 lines of headers
            max_match = 0 + max_match         # convert from string to integer
            max_lines = max_lines + n_headers # max output lines
            n_ignore = 0
        }
        
        $0 ~ pattern {
            print_clear_ring_buff(ring_buff)
            print; ++n_match; tailing = 5; next;
        }

        FNR < n_headers { print; next; }
        FNR > max_lines { print "[WORNING] message overflow, FNR=" FNR; exit; }
        tailing > 0 { tailing -= 1; print; next; }
        n_match >= max_match { exit }
        
        { enque_ring_buff(ring_buff) }
        ' $DEBUGFS/tracing/trace
)

### <a id='using_trace_printk'>2.4 Using `trace_printk()`</a>

C function `trace_printk()` has much better performance than `printk()`, and it writes to the ring buffer of `trace` file. It can be used just like `printk()`, and can also be used in any context (interrupt code, NMI code, and scheduler code).

I wrote a _Hello World_ of ftrace, the main part is showed below:

```c
// file: hello_ftrace/hello_ftrace.c
static int hello_init(void)
{
    trace_printk("Hello, Ftrace!\n");
    return 0;
}
static void hello_exit(void)
{
    trace_printk("Good bye, Ftrace!\n");
}
```

In [15]:
make -C hello_ftrace

make: Entering directory '/home/clymber/Studio/kerdevel/ftrace/hello_ftrace'
make -C /lib/modules/6.1.30/build M=/home/clymber/Studio/kerdevel/ftrace/hello_ftrace modules
make[1]: Entering directory '/home/clymber/@linux'
make[1]: Leaving directory '/home/clymber/@linux'
make: Leaving directory '/home/clymber/Studio/kerdevel/ftrace/hello_ftrace'


In [16]:
echo > $DEBUGFS/tracing/trace # clear ring buffer first
insmod hello_ftrace/hello_ftrace.ko    # install module
rmmod hello_ftrace                     # remove module
filter_trace 'Ftrace'

# tracer: nop
#
# entries-in-buffer/entries-written: 2/2   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |
          insmod-7869    [007] .....  6431.189376: 0xffffffffc2071005: Hello, Ftrace!
           rmmod-7871    [000] .....  6431.243673: 0xffffffffc2071035: Good bye, Ftrace!


In the output above, we can see that the current tracer is `nop`, which usually means the tracing is currently off.  
According to [Debugging the kernel using Ftrace - part 1](https://lwn.net/Articles/365835/),`trace_printk()` output will appear in any tracer, even the function and function graph tracers.

In [17]:
echo 'function_graph' > $DEBUGFS/tracing/current_tracer

modprobe hello_ftrace    # install module
rmmod hello_ftrace       # remove module
filter_trace 'Ftrace'
echo 'nop' > $DEBUGFS/tracing/current_tracer

# tracer: function_graph
#
# CPU  DURATION                  FUNCTION CALLS
# |     |   |                     |   |   |   |
 4)               |                  reweight_entity() {
 1)   0.093 us    |            } /* __rcu_read_unlock */
 1)   4.834 us    |          } /* kmem_cache_alloc */
 1)               |          mm_init() {
 1)   0.090 us    |            __init_rwsem();
 1)               |            pgd_alloc() {
 1)               |              __get_free_pages() {
[WORNING] message overflow, FNR=1013


Oops, There is no 'Hello, Ftrace!' nor 'Good bye, Ftrace' found in the output above! Why was it? Maybe it's because the kernel is too busy that the ring buffer was overflow over and over again.

### <a id='ring_buff_overflow'>2.5 Ring Buffer Overflow Problem</a>

For the problem I found at the last section, I found the answer in [Debugging the kernel using Ftrace - part 2](https://lwn.net/Articles/366796/). The solution is: within the kernel, turn on/off the tracing feature only when it's necessary.

Alright, let's try it by modifying the _Hello World_ a little bit:

```sh
awk '/^static/,/^}$/' hello_ftrace/hello_ftrace.c
```
```c
// file: hello_ftrace/hello_ftrace.c
static int hello_init(void)
{
    tracing_on(); // <---- new
    trace_printk("Hello, Ftrace!\n");
    tracing_off(); // <---- new
    return 0;
}
static void hello_exit(void)
{
    tracing_on(); // <---- new
    trace_printk("Good bye, Ftrace!\n");
    tracing_off(); // <---- new
}
```

In [11]:
make -C hello_ftrace

make: Entering directory '/home/clymber/Studio/kerdevel/ftrace/hello_ftrace'
make -C /lib/modules/6.1.30/build M=/home/clymber/Studio/kerdevel/ftrace/hello_ftrace modules
make[1]: Entering directory '/home/clymber/@linux'
  CC [M]  /home/clymber/Studio/kerdevel/ftrace/hello_ftrace/hello_ftrace.o
  MODPOST /home/clymber/Studio/kerdevel/ftrace/hello_ftrace/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/ftrace/hello_ftrace/hello_ftrace.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/ftrace/hello_ftrace/hello_ftrace.ko
make[1]: Leaving directory '/home/clymber/@linux'
make: Leaving directory '/home/clymber/Studio/kerdevel/ftrace/hello_ftrace'


In [13]:
echo 'function_graph' > $DEBUGFS/tracing/current_tracer
echo '0' > $DEBUGFS/tracing/tracing_on

insmod hello_ftrace/hello_ftrace.ko    # install module
rmmod hello_ftrace       # remove module
filter_trace 'Ftrace'

echo 'nop' > $DEBUGFS/tracing/current_tracer

# tracer: function_graph
#
# CPU  DURATION                  FUNCTION CALLS
# |     |   |                     |   |   |   |
 0)               |  /* Hello, Ftrace! */
 ------------------------------------------
 0)  insmod-10175  =>  rmmod-10177  
 ------------------------------------------

 0)               |  /* Good bye, Ftrace! */


It worked! Let's try `function` tracer and have a look:

In [14]:
echo 'function' > $DEBUGFS/tracing/current_tracer

insmod hello_ftrace/hello_ftrace.ko    # install module
rmmod hello_ftrace       # remove module
filter_trace 'Ftrace'

echo 'nop' > $DEBUGFS/tracing/current_tracer

# tracer: function
#
# entries-in-buffer/entries-written: 2/2   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |
          insmod-10199   [004] ..... 10808.424001: 0xffffffffc2071005: Hello, Ftrace!
           rmmod-10201   [000] ..... 10808.482073: 0xffffffffc2071045: Good bye, Ftrace!


Things turned out very well.  
But there's one thing interesting that got my attention: the IDs of processes that printed 'Hello, Ftrace!' and 'Good bye, Ftrace!' were different, 10199 and 10201 respectively, although the functions called are within the same kernel module, *hello_ftrace.ko*. I think it's because the modules which loaded in kernel are not running within a specific process.

Each kernel module in the memory represents a shared object that contains symbols which processes can reference to, just like a shared library('xxx.so') in the user space does. I'll test and verify it in another experiment: [Get Process ID from a Kernel Module](../module_getpid/get_pid_from_kernel.ipynb).

### <a id='start_stop_recording'>2.6 Starting and Stopping the Trace Recording</a>

Let's learn a bit more about how to control the trace recording feature.
The file `tracing_on` is used to disable/enable the ring buffer from recording data:
```sh
echo '0' > tracing_on   # stop  tracing
echo '1' > tracing_off  # start tracing 
```
Note that don't confound the commands above with I/O redirection of shell script. The following commands will not work for the Ftrace:
```sh
echo 0> tracing_on  # redirect STDIN to tracing_on
echo 1> tracing_off # redirect STDOUT to tracing_on
```
There are two functions that work well inside the kernel: `tracing_on()` and `tracing_off()`. These two act just like echoing "1" or "0" respectively into the tracing_on file.
```c
void tracing_on(void);
void tracing_off(void);
```
Writing '0' into *tracing_on* will disable the Ftrace ring buffer from recording. Everything else still happens with the tracers and they will still incur most of their overhead. They do notice that the ring buffer is not recording and will not attempt to write any data, but the calls that the tracers make are still performed.  
If we want to disable the ftrace, we should use the file *current_tracer* instead of *tracing_on*, see [Enable/Disable Ftrace](#enable_disable).